<a href="https://colab.research.google.com/github/rg-smith/remote-sensing-hydro-2025/blob/main/assignments/homework2/homework2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework 2

In this lab, you will use python (and a module called Google Earth Engine) to view satellite imagery, and access precipitation datasets.

Before starting the sections, we will import the modules we need, initialize Google Earth Engine, and define some functions.

The first block of code only needs to be run once, the first time you open your session (if you close out of the session and open again, you will need to run this again). Follow the prompts to initialize earth engine. You will be taken to a link where you need to give permission to link your google account with Google Earth Engine, then copy and paste some text below the code block.

In [ ]:
import ee
import folium
import numpy as np
import branca.colormap as cm

from google.colab import drive
drive.mount('/content/drive/')

If you have not registered for Google Earth Engine yet, do so at https://code.earthengine.google.com/. At the top-right corner, you will see a name next to your user icon. Copy that name and past where 'your project name' is located in the code block below. NOTE: if you see any warning symbols on the code.earthengine.google.com website next to your profile icon, click it and resolve it before proceeding.

In [11]:
#if not ee.data._credentials:
ee.Authenticate()
ee.Initialize(project='geocode-1322')

This block of code also only needs to be run once. It defines many useful functions, some of which you will use in this exercise. The most important thing for you to understand is what arguments are needed to run the functions, and what their outputs are. You will use some of them in this exercise, and some later on!

In [14]:
# functions needed for this lab (and some other useful ones that you can use if you're interested)

# this function is used to add a google earth engine layer to an existing folium map,
# for visualization purposes. Folium is a python package that can put rasters/shapefiles on a basemap
# the function below is run using an existing folium map. If the folium map defines is my_map, then
# my_map.add_ee_layer(ee_object,name)
# where ee_object is the object defined in google earth engine, and name is the label in folium
def add_ee_layer(self, ee_object, name):
    try:
        # display ee.Image()
        if isinstance(ee_object, ee.image.Image):
            range = ee.Image(ee_object).reduceRegion(ee.Reducer.percentile([1, 99]),scale=10000)
            vals = range.getInfo()
            min=list(vals.items())[0][1]
            max=list(vals.items())[1][1]
            vis = {'min': min, 'max': max, 'palette': ['0000FF', 'FFFFFF','FF0000']}

            map_id_dict = ee.Image(ee_object).getMapId(vis)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
            colormap = cm.LinearColormap(vmin=min,vmax=max,colors=['blue', 'white','red']).to_step(n=10)
            colormap.caption=name
            self.add_child(colormap)
        # display ee.ImageCollection()
        elif isinstance(ee_object, ee.imagecollection.ImageCollection):
            ee_object_new = ee_object.mosaic()
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.Geometry()
        elif isinstance(ee_object, ee.geometry.Geometry):
            folium.GeoJson(
            data = ee_object.getInfo(),
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
        # display ee.FeatureCollection()
        elif isinstance(ee_object, ee.featurecollection.FeatureCollection):
            ee_object_new = ee.Image().paint(ee_object, 0, 2)
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
        ).add_to(self)

    except Exception as e:
        print("Could not display {}".format(name))
        print(e)

# to convert a google earth engine image to a python array
def to_array(img,aoi):
  band_arrs = img.sampleRectangle(region=aoi,properties=['scale=1000'],defaultValue=-999)

  band_names=img.bandNames().getInfo()

  for kk in range(len(band_names)):
    if kk==0:
      dat1=np.array(band_arrs.get(band_names[kk]).getInfo())
      dat_full=np.zeros((dat1.shape[0],dat1.shape[1],len(band_names)))
      dat_full[:,:,kk]=dat1
    else:
      dat=np.array(band_arrs.get(band_names[kk]).getInfo())
      dat_full[:,:,kk]=dat
  return(dat_full)

# to calculate an index
def getIndex(image,b1,b2):
  return image.normalizedDifference([b1, b2])

# to calculate a ratio
def getRatio(image1,image2):
  ratio=image1.divide(image2)
  return ratio

# to create a color map from a specific image
def getVisparams(image,aoi):
  range = image.reduceRegion(ee.Reducer.percentile([1, 99]),aoi,300)
  vals = range.getInfo()
  min=list(vals.items())[0][1]
  max=list(vals.items())[1][1]
  visParams = {'min': min, 'max': max, 'palette': ['0000FF', 'FFFFFF','FF0000']}
  return(visParams)

# to get the link to download an earth engine image
def getLink(image,aoi):
  link = image.getDownloadURL({
    'scale': 1000,
    'crs': 'EPSG:4326',
    'fileFormat': 'GeoTIFF',
    'region': aoi})
  print(link)

# create an earth engine geometry polygon
def addGeometry(min_lon,max_lon,min_lat,max_lat):

  geom = ee.Geometry.Polygon(
      [[[min_lon, max_lat],
        [min_lon, min_lat],
        [max_lon, min_lat],
        [max_lon, max_lat]]])
  return(geom)

# load prism data
def get_prism_image(date1,date2,geometry):

  prism = ee.ImageCollection('OREGONSTATE/PRISM/AN81m')
  prism_img = prism.filterDate(date1,date2).select('ppt').mean().clip(geometry)
  return(prism_img) # returns prism average monthly precipitation, in mm

# load landsat 8 data
def get_l8_image(date1,date2,geometry):

  l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_RT')
  l8_img = l8.filterDate(date1,date2).mean().clip(geometry)
  return(l8_img)

# to export an image to google drive
def export_to_drive(raster,filename,foldername,geometry):
  # Export the image, specifying scale and region.
  task = ee.batch.Export.image.toDrive(**{
      'image': raster,
      'description': filename,
      'folder': foldername,
      'fileNamePrefix': filename,
      'scale': 1000,
      'region': geometry,
      'fileFormat': 'GeoTIFF',
      'formatOptions': {
        'cloudOptimized': 'true'
      },
  })
  task.start()

# to create an elevation raster from the USGS NED in google earth engine from a user-defined geometry
def get_elev(geometry):

  elev = ee.Image('USGS/NED').clip(geometry)
  return(elev)

# to create an elevation raster from the SRTM in google earth engine from a user-defined geometry
def get_srtm(geometry):

  elev = ee.Image('USGS/SRTMGL1_003').clip(geometry)
  return(elev)

# to create a temporally averaged precipitation raster from GPM from a user-defined geometry
def get_gpm_image(date1,date2,geometry):

  gpm = ee.ImageCollection('NASA/GPM_L3/IMERG_MONTHLY_V07')
  gpm_img = gpm.filterDate(date1,date2).select('precipitation').mean().multiply(24*365/12).clip(geometry) # convert from mm/hour to mm/month
  return(gpm_img) # returns gpm average monthly precipitation in mm

# to create a temporally averaged actual ET raster from the openET ensemble from a user-defined geometry
def get_openET_image(date1,date2,geometry):

  openET = ee.ImageCollection('OpenET/ENSEMBLE/CONUS/GRIDMET/MONTHLY/v2_0')
  openET_img = openET.filterDate(date1,date2).select('et_ensemble_mad').mean().clip(geometry)
  return(openET_img)

# to create a temporally averaged reference ET raster from the openET ensemble from a user-defined geometry
def get_RET(date1,date2,geometry):

  ETR = ee.ImageCollection('IDAHO_EPSCOR/GRIDMET')
  ETR_image = ETR.filterDate(date1,date2).select('etr').mean().multiply(365/12).clip(geometry) # convert from mm/day to mm/month
  return(ETR_image)

# load sentinel 2 data
def get_s2_image(date1,date2,geometry):

    s2 = ee.ImageCollection('COPERNICUS/S2')
    s2_img = s2.filterDate(date1,date2).filterBounds(geometry).first().clip(geometry)
    return(s2_img)

# Add EE drawing method to folium (not a function)
folium.Map.add_ee_layer = add_ee_layer

# Part 1: Viewing precipitation data for a given geometry and date



First, we will define the study area and time period and load some PRISM, GPM and elevation data.

In [15]:
# create a bounding box that defines the study area
geom = addGeometry(-109, -102,37,41) # min long, max long, min lat, max lat

# define dates of interest (inclusive).
start = '2021-10-01'
end = '2022-10-01' #can go up to april 2021

# get elevation data
elev = get_elev(geom)

# now get gpm precipitation over the same region for a specified time period
gpm_img = get_gpm_image(start,end,geom)

# now get prism precipitation over the same time period/region
prism_img = get_prism_image(start,end,geom)

# now get actual ET from openET over the same time period/region
openET_img = get_openET_image(start,end,geom)

# now get reference ET
RET = get_RET(start,end,geom)

Now, we will create the map. We have not displayed it yet, so you will not see it after running this code block.

In [16]:
my_map = folium.Map(location=[39, -104.5], zoom_start=7)

# Add the land cover to the map object.
#my_map.add_ee_layer(l8_img,'Landsat 8')
my_map.add_ee_layer(geom,'bounding box')
my_map.add_ee_layer(elev,'Elevation (m)')
my_map.add_ee_layer(prism_img,'PRISM precip (mm/month)')
my_map.add_ee_layer(gpm_img,'GPM precip (mm/month)')
my_map.add_ee_layer(openET_img,'ET (openET, mm/month)')



Now we will compare monthly ET to monthly precipitation. Areas with much higher ET than precipitation could receive water from streams, groundwater, or irrigation. They could also represent regions with errors in the precipitation or ET estimates.

We will also divide monthly actual ET by monthly reference ET.

The last line displays the map. Scroll to the top so you can check/uncheck layers. Note that the bottom layer on the legend will show up on top. Take a screenshot for your lab report. Comment on the similarities and discrepancies you see between GPM and PRISM precipitation. Do you notice any trends in the discrepancies?

Comment also on discrepancies between ET and precipitation. Group them into discrepancies that you think have a physical meaning, and discrepancies that you think may be related to noise in the data. Be thoughtful about this! 'It looks noisy' is not a sufficient response!

Also comment on the ratio (actual ET/reference ET). What does that ratio physically represent?

In [17]:
ET_precip = openET_img.subtract(prism_img)
my_map.add_ee_layer(ET_precip,'ET - precip (mm/month)')

ET_ETR = openET_img.divide(RET)
my_map.add_ee_layer(ET_ETR,'ET / ETR')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)

Now, we will export the tiffs to your Google Colab folder (this is a folder in your Google Drive storage). We will then use some other python functions to do analysis. After you run the code below, it will take a few minutes to save the files to your Google Drive. Move on to the next step while you're waiting.

In [ ]:
export_to_drive(gpm_img,'lab2_gpm_precip_CO','Colab Notebooks',geom) # raster, file name, folder name, geometry to clip it with
export_to_drive(elev,'lab2_elev_CO','Colab Notebooks',geom)
export_to_drive(prism_img,'lab2_prism_precip_CO','Colab Notebooks',geom)

While you're waiting, try a different way of downloading the rasters below. Once you have downloaded them, load them into QGIS. Reproject the elevation dataset to a projected coordinate reference system (UTM Zone 15N, WGS84 is a good choice). You can do this by right-clicking on the elevation file, Export, then specify the coordinate reference system.

Calculate the slope of the projected coordinate reference system (Raster -> Analysis -> Slope).

Identify an area with high landslide risk (high slope/high precipitation), and make a shapefile  over that region.

Take a screenshot and include with your report.

The precipitation units are the average monthly precipitation in mm. What is the maximum/minimum total precipitation over the study period?

In [ ]:
print('GPM precipitation raster download:')
getLink(gpm_img,geom)
print('Elevation raster download:')
getLink(elev,geom)
print('PRISM precipitation raster download:')
getLink(prism_img,geom)

Now, we will load the rasters we just downloaded with a separate python package--rasterio--and compare GPM with PRISM.

In [ ]:
!pip install rasterio
import rasterio
from rasterio.warp import reproject
import matplotlib.pyplot as plt

Click on the 'Files' tab on the left. Go to drive > MyDrive > Colab Notebooks. You should see some '.tif' files for prism, gpm and elevation data. If you don't see them yet, you may have to wait a few minutes for them to get saved to the cloud.

Now you will load the rasters into python with rasterio. This package works more easily with the usual python packages, making plotting easier.

In [34]:
folder_location='drive/MyDrive/Colab Notebooks/'
gpm=rasterio.open(folder_location+'lab2_gpm_precip_CO.tif')
prism=rasterio.open(folder_location+'lab2_prism_precip_CO.tif')
elev=rasterio.open(folder_location+'lab2_elev_CO.tif')

Now, we will make a crossplot of PRISM and GPM data, with points colored by elevation. PRISM are considered more accurate generally. Comment on the accuracy of the GPM data.

In [ ]:
gpm_flat=gpm.read().flatten()
prism_flat=prism.read().flatten()
elev_flat=elev.read().flatten()
plt.figure(figsize=(12,8));plt.scatter(gpm_flat,prism_flat,c=elev_flat,s=1)
reference_line=[np.nanmin(gpm_flat),np.nanmax(gpm_flat)]
plt.plot(reference_line,reference_line,'r')
plt.xlabel('GPM precipitation, mm/month')
plt.ylabel('PRISM precipitation, mm/month')
plt.colorbar()

# Part 2: Repeat the above process, but this time do it over the study area for your term project

If you haven't decided on a location for your project yet, you can select an area you are considering for your term project. It's OK to change later.

You will need to 'fill in the blanks' for the code below.

In [ ]:
# create a bounding box that defines the study area. This will cover the Lake of the Ozarks region
geom_study_area = addGeometry(?,?,?,?) # min long, max long, min lat, max lat

# define dates of interest (inclusive).
start = '?'
end = '?'

# use the tools in the example from Part 1 to pull GPM precip and elevation data.
# NOTE: if you are workin in a region outside of the US, the get_elev function will not work
# (it uses the National Elevation Dataset which only covers the US). Use the get_srtm function instead

Now, you will display the map. Include a screenshot in your lab report.

In [ ]:
my_map = folium.Map(location=[?, ?], zoom_start=11) # location values should be lat, lon. zoom_start: lower values are more zoomed out. Adjust until it looks right.

# Add the precip, ET and elevation to your map object
my_map.add_ee_layer(geom_study_area,'bounding box')
my_map.add_ee_layer(?,'? label')
my_map.add_ee_layer(?,'? label')
my_map.add_ee_layer(?,'? label')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)

Download these rasters and display in QGIS as you did before. Repeat the same exercise -- reproject the elevation to a projected CRS, then calculate slope and identify an area of high landslide risk.

As before, determine the average precipitation over the study period.

In [ ]:
print('GPM precipitation raster download:')
getLink(?,geom_study_area)
print('Elevation raster download:')
getLink(?,geom_study_area)
print('PRISM precipitation raster download')
getLink(?,geom)

If you'd like to get more familiar with python, try creating a new code block to do the same analysis as before (load the rasters with rasterio, then make a crossplot). It's mostly just copying code from above, and changing filenames. This part is not required, but if you try it, you will get more comfortable with python.